## Causal Language Model

In [ ]:
%%writefile requirements.txt
torch
pytorch-ignite
transformers
datasets
evaluate
accelerate

Writing requirements.txt


In [ ]:
! pip install -r requirements.txt

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00


### Load Dataset
Data: `eli5_category`

Model: `distilbert/distilgpt2`

In [ ]:
from datasets import load_dataset

dataset = load_dataset("eli5_category", split="train[:5000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


eli5_category.py:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

The repository for eli5_category contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eli5_category.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 1000
    })
})

In [ ]:
new_dataset = dataset['train'].train_test_split(test_size=0.05)
dataset['train'] = new_dataset['train']
dataset['validation'] = new_dataset['test']

In [ ]:
dataset = dataset.flatten()
dataset['train'][0]

{'q_id': '5n3pxu',
 'title': 'If energy can neither be created nor destroyed; only converted, what energy is converted to kinetic energy of a pin that accelerates towards a magnet as a magnet approaches it (assuming level surface, of course)',
 'selftext': '',
 'category': 'Physics',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dc8g6ag', 'dc8kdf0'],
 'answers.text': ['Potential energy. Someone put the magnet and pin close to another but seperated. Thus he transformed kinetic energy into potential energy. Which then is re transformed by the magnet into kinetic energy accelerating the pin towards it. If you would separate the pin from the magnet again you would then transform more kinetic energy back to potential energy.',
  "Magnetic potential energy. The magnet and the pin (combined) have some energy just by existing and being far apart. When they move closer together this potential energy is converted to kinetic energy. When you pull them apart again, you're supplying kinetic

### Preprocessing

In [ ]:
from transformers import AutoTokenizer
model_name = "distilbert/distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples['answers.text']])

examples = dataset['train'][:5]
examples = preprocess_function(examples)
examples[:3]

[Encoding(num_tokens=155, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=585, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=84, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [ ]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset['train'].column_names
)

Map (num_proc=4):   0%|          | 0/3800 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1906 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1469 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1129 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1417 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1956 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1758 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1676 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1029 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2167 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1509 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2026 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3800
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 200
    })
})

In [ ]:
block_size = 128

def group_texts(examples):
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])

  if total_length >= block_size:
    total_length = (total_length // block_size) * block_size

  result = {
      k: [t[i:i+block_size] for i in range(0, total_length, block_size)]
      for k, t in concatenated_examples.items()
  }
  result["labels"] = result["input_ids"].copy()
  return result

examples = tokenized_datasets['train'][:5]
examples = group_texts(examples)
examples.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=4,
)

lm_datasets.set_format("torch")

Map (num_proc=4):   0%|          | 0/3800 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
train_dataset = lm_datasets['train']
eval_dataset = lm_datasets['validation']

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(eval_dataset, batch_size=8)

In [ ]:
from transformers import AutoModelForCausalLM
from accelerate import Accelerator
from torch.optim import AdamW
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup
import torch
from torch.nn import CrossEntropyLoss


epochs = 2
# accelerator = Accelerator()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModelForCausalLM.from_pretrained(model_name)
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
loss_fct = CrossEntropyLoss()


scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=len(train_dataloader) * epochs
)

# model, optimizer, train_dataloader, scheduler = accelerator.prepare(
#     model, optimizer, train_dataloader, scheduler
# )

model = model.to(device)


In [ ]:
from multiprocessing import process
for epoch in range(epochs):
  model.train()
  progress_bar = tqdm(train_dataloader)
  for batch in progress_bar:
    optimizer.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    outputs = outputs.logits

    loss = loss_fct(outputs.view(-1, outputs.shape[-1]), labels.view(-1))

    # accelerator.backward(loss)
    loss.backward()
    optimizer.step()
    scheduler.step()
    progress_bar.set_description(f"Training loss: {loss.item():.4f}, Epoch: {epoch}")

  val_progress_bar = tqdm(eval_dataloader)
  model.eval()
  for batch in val_progress_bar:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    with torch.no_grad():
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      outputs = outputs.logits
      loss = loss_fct(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
    val_progress_bar.set_description(f"Validation loss: {loss.item():.4f}, Epoch: {epoch}")

  print(f"Epoch: {epoch}, Loss: {loss.item()}")




  0%|          | 0/1239 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

Epoch: 0, Loss: 0.05204712226986885


  0%|          | 0/1239 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

Epoch: 1, Loss: 0.04554321616888046


In [ ]:
batch = next(iter(train_dataloader))
batch = {k: v.to(device) for k, v in batch.items()}
input_ids = batch['input_ids']
attention_mask = batch['attention_mask']
labels = batch['labels']



with torch.no_grad():
  out = model(input_ids, attention_mask=attention_mask)
  out = out.logits
  out = out.argmax(dim=-1)

  # loss = loss_fct(out.view(-1, out.shape[-1]), labels.view(-1))

  # print(loss)
  print(out)
  out = tokenizer.batch_decode(out)
  print(out)
  labels = tokenizer.batch_decode(labels)
  print(labels)

tensor([[  262,   345,   836,  ...,   843,   428,   318],
        [   11,  1243,   460,  ...,    13,   317,  1884],
        [   11,   531,   262,  ...,   284,  6901,   262],
        ...,
        [   11,   588,   366,  ..., 14990,   290, 41051],
        [  262,  1049,   890,  ...,  1688,  5410,  2458],
        [   13,  4329,   329,  ...,   340,   481,  6209]], device='cuda:0')
[" the you don't lose the water that you would. But since alcohol slows/inhibits this, it means your body doesn't use the loop of henle as well, as you don't get as much water OUT of the urine and staying in your body. So, if your body doesn't get as much water back...you pee more out! Which makes you have more to pee, and WHAT you pee is less concentrated (so it's pretty clear and less yellow). But that also means that your body kept less water inside...and that means that usually the next day, you wake up feeling dehydrated! And this is", ", things can happen. Wouldn't take too long to realize something importan

torch.Size([8, 128])

In [ ]:
model_save = "distilgpt2_CasualLM_eli5"
model.save_pretrained(model_save)
tokenizer.save_pretrained(model_save)

('distilgpt2_CasualLM_eli5/tokenizer_config.json',
 'distilgpt2_CasualLM_eli5/special_tokens_map.json',
 'distilgpt2_CasualLM_eli5/vocab.json',
 'distilgpt2_CasualLM_eli5/merges.txt',
 'distilgpt2_CasualLM_eli5/added_tokens.json',
 'distilgpt2_CasualLM_eli5/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub(model_save)
tokenizer.push_to_hub(model_save)

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/PradhyumnaPoralla/distilgpt2_CasualLM_eli5/commit/0e2268b451660c8a753b03f400d191eff81a8826', commit_message='Upload tokenizer', commit_description='', oid='0e2268b451660c8a753b03f400d191eff81a8826', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
prompt = "Someone put the magnet and pin close"

tokenizer = AutoTokenizer.from_pretrained("PradhyumnaPoralla/distilgpt2_CasualLM_eli5")
model = AutoModelForCausalLM.from_pretrained("PradhyumnaPoralla/distilgpt2_CasualLM_eli5")

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(input_ids, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.98)

print(tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0])

# dataset['train'][0]['answers.text'][0][:50]

# input = tokenizer(prompt, return_tensors='pt').to(device)

# with torch.no_grad():
#   out = model(**input)
#   out = out.logits
#   out = out.argmax(dim=-1)
#   out = tokenizer.batch_decode(out)
#   print(out)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Someone put the magnet and pin close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close close
